In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
from collections import Counter
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import seaborn as sns

In [2]:
import pyLDAvis
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
import pyLDAvis.gensim_models as gensimvis
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample
from pprint import pprint
from gensim.models import CoherenceModel

C:\Users\Stella\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
C:\Users\Stella\Anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps
C:\Users\Stella\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
dataset = pd.read_csv("D:/Stella/Documents/10_Academy/Twitter-Data-Analysis/data/covid19.csv")


In [4]:
tweets_text=dataset[['clean_text']]
tweets_text.head(2)

,clean_text
0,"['africa', '""in', 'midst', 'full-blown', 'thir..."
1,"['dr', 'moeti', 'head', 'africa', 'one', 'best..."


In [5]:
words = [[text for text in doc.split()] for doc in tweets_text['clean_text']]
#words

In [6]:
# Create Dictionary
covid_dict = corpora.Dictionary(words)

# Create Corpus
texts = words

# Term Document Frequency
covidcorpus = [covid_dict.doc2bow(text) for text in texts]

# View
#print(corpus[:1][0][:10])

In [7]:
num_topics=10
# Build LDA model
lda_model = models.LdaMulticore(corpus=covidcorpus,
                                       id2word=covid_dict,
                                        alpha=0.01,
                                        num_topics=num_topics, 
                                        random_state=100)

# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[covidcorpus]

[(0,
  '0.046*"[\'nan\']" + 0.028*"\'vaccine\'," + 0.026*"\'#covid\'," + '
  '0.011*"\'hospital\'," + 0.010*"\'day\'," + 0.009*"\'amp\'," + '
  '0.009*"\'case\'," + 0.009*"\'%\'," + 0.007*"\'world\'," + 0.007*"\'new\',"'),
 (1,
  '0.071*"[\'nan\']" + 0.037*"\'vaccine\'," + 0.022*"\'#covid\'," + '
  '0.014*"\'dos\'," + 0.011*"\'world\'," + 0.011*"\'amp\'," + '
  '0.009*"\'million\'," + 0.008*"\'africa\'," + 0.007*"\'%\'," + '
  '0.007*"\'pfizer\',"'),
 (2,
  '0.080*"[\'nan\']" + 0.042*"\'vaccine\'," + 0.028*"\'#covid\'," + '
  '0.013*"\'people\'," + 0.013*"\'india\'," + 0.010*"\'amp\'," + '
  '0.009*"\'call\'," + 0.009*"\'vaccinated\'," + 0.009*"\'today\'," + '
  '0.008*"\'find\',"'),
 (3,
  '0.037*"\'vaccine\'," + 0.026*"\'#covid\'," + 0.019*"\'dos\'," + '
  '0.013*"\'amp\'," + 0.013*"\'million\'," + 0.012*"\'case\'," + '
  '0.012*"\'third\'," + 0.010*"\'%\'," + 0.009*"[\'nan\']" + '
  '0.008*"\'people\',"'),
 (4,
  '0.070*"\'vaccine\'," + 0.036*"\'#covid\'," + 0.010*"\'covid\'," + '
 

In [8]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(covidcorpus))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=words, dictionary=covid_dict, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -6.271788964120024

 Ldamodel Coherence Score/Accuracy on Tweets:  0.5427596798335222


In [9]:
#"""Save model objects"""
SaveLoad.save(lda_model,'covidLDAmodel')
corpora.MmCorpus.serialize('covidCorpus.mm', covidcorpus)
covid_dict.save('covidDictionary.dict')

In [10]:
import pyLDAvis
pyLDAvis.enable_notebook()
ldaViz = gensimvis.prepare(lda_model, covidcorpus, covid_dict)

In [11]:
ldaViz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.140687  0.141189       1        1  16.968866
4      0.015281 -0.080982       2        1  16.938825
2     -0.043874 -0.100739       3        1  12.677377
5     -0.052730  0.134747       4        1  12.488697
7      0.304189  0.086668       5        1  10.374757
3     -0.018341 -0.042980       6        1   8.057527
1     -0.019956  0.045363       7        1   5.977662
8     -0.003002 -0.093654       8        1   5.824722
0     -0.056857  0.017981       9        1   5.481123
9      0.015976 -0.107593      10        1   5.210444, topic_info=              Term         Freq        Total Category  logprob  loglift
183        ['nan']  3314.000000  3314.000000  Default  30.0000  30.0000
20       'africa',   688.000000   688.000000  Default  29.0000  29.0000
68       '#covid',  1863.000000  1863.000000  Default  28.0000  28.0000
32          'one',   454.000000   454.000000  Default  27.0000  27.0000
31        'moeti',   185.000000   185.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
94         'help',    15.825348   168.775550  Topic10  -5.6139   0.5875
210        'risk',    15.237782   147.832038  Topic10  -5.6517   0.6822
88        'covid',    19.113848   483.087363  Topic10  -5.4251  -0.2753
290       'world',    19.606897   681.943846  Topic10  -5.3996  -0.5946
212  'vaccinated',    16.004006   333.860770  Topic10  -5.6026  -0.0834

[713 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
0         1  0.612693           '"in',
0         2  0.042255           '"in',
0         4  0.218316           '"in',
0         6  0.021127           '"in',
0         7  0.035212           '"in',
...     ...       ...              ...
3711      3  0.033196  ['vaccination',
3711      5  0.033196  ['vaccination',
3711      8  0.431546  ['vaccination',
3711      9  0.132783  ['vaccination',
1059      5  0.884054      ['waiting',

[2648 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 3, 6, 8, 4, 2, 9, 1, 10])